In [1]:
import darkwing as dw

def foo(rel, res=6):
    return (rel
    | 'select pickup_latitude as lat, pickup_longitude as lng, tip_amount'       
    | f'select h3_latlng_to_cell(lat, lng, {res}) as hexid, tip_amount as tip'
    | 'select hexid, avg(tip) as tip group by 1'
    | 'select h3_h3_to_string(hexid) as hexid, tip'
    | 'where tip > 0'
    )

In [2]:
# out = dw.query('select * from "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet"')

In [3]:
rel1 = dw.load('data/yellow_tripdata_2010-01.parquet') # lazy load!
rel2 = dw.load('data/yellow_tripdata_2010-02.parquet') # lazy load!

In [4]:
rel1 | 'select arbitrary(columns(*))' | dict

{'vendor_id': 'VTS',
 'pickup_datetime': '2010-01-26 07:41:00',
 'dropoff_datetime': '2010-01-26 07:45:00',
 'passenger_count': np.int64(1),
 'trip_distance': np.float64(0.75),
 'pickup_longitude': np.float64(-73.956778),
 'pickup_latitude': np.float64(40.76775),
 'rate_code': '1',
 'store_and_fwd_flag': None,
 'dropoff_longitude': np.float64(-73.965957),
 'dropoff_latitude': np.float64(40.765232),
 'payment_type': 'CAS',
 'fare_amount': np.float64(4.5),
 'surcharge': np.float64(0.0),
 'mta_tax': np.float64(0.5),
 'tip_amount': np.float64(0.0),
 'tolls_amount': np.float64(0.0),
 'total_amount': np.float64(5.0)}

In [5]:
rel1 >> 'select count()' >> int

14863778

In [6]:
# TODO: why isn't this lazy?
# rel3 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet' >> dw.load

In [7]:
rel4 = 'data/yellow_tripdata_2010-01.parquet' >> dw.load

In [8]:
rel1 | foo

┌─────────────────┬────────────────────┐
│      hexid      │        tip         │
│     varchar     │       double       │
├─────────────────┼────────────────────┤
│ 862a100d7ffffff │ 0.6042399096361921 │
│ 862a1008fffffff │ 0.5471456491468882 │
│ 86754e64fffffff │ 0.6730361779149294 │
│ 862a10727ffffff │ 0.6621288459090504 │
│ 862a100e7ffffff │ 1.9770422191887693 │
│ 862a10117ffffff │  6.559024390243902 │
│ 862a10387ffffff │ 1.5341203703703705 │
│ 862a10a97ffffff │  2.658620689655172 │
│ 862a1031fffffff │ 1.2615384615384615 │
│ 862a30297ffffff │ 1.5257142857142856 │
│        ·        │                 ·  │
│        ·        │                 ·  │
│        ·        │                 ·  │
│ 86eac3757ffffff │               4.92 │
│ 862a14977ffffff │                2.0 │
│ 862b89367ffffff │                0.5 │
│ 862b88a1fffffff │                1.5 │
│ 862ba959fffffff │               1.75 │
│ 862a31887ffffff │                1.0 │
│ 862b899b7ffffff │                2.6 │
│ 862aaccdffffff

In [9]:
out = dw.Database(
    rel1 = rel1 >> foo,
    rel2 = rel2 >> foo,
)
out

Database:
    rel1: 850 x ['hexid', 'tip']
    rel2: 731 x ['hexid', 'tip']

In [11]:
out = dw.Database(
    rel1 = rel1 >> foo,
    rel2 = rel2 >> foo,
) | """
select
    hexid,
    rel1.tip
        as tip1,
    rel2.tip
        as tip2,
from rel1
full outer join rel2
using (hexid)
"""

(out
| 'where tip1 is not null and tip2 is not null'
| 'select *, tip1-tip2 as diff order by diff'
) >> 'as bam' # precedence rules!

Database:
    bam: 371 x ['hexid', 'tip1', 'tip2', 'diff']

In [10]:
rel1 = dw.load('data/yellow_tripdata_2010-01.parquet')

In [8]:
rel1._rel

┌───────────┬─────────────────────┬─────────────────────┬───┬────────────┬──────────────┬──────────────┐
│ vendor_id │   pickup_datetime   │  dropoff_datetime   │ … │ tip_amount │ tolls_amount │ total_amount │
│  varchar  │       varchar       │       varchar       │   │   double   │    double    │    double    │
├───────────┼─────────────────────┼─────────────────────┼───┼────────────┼──────────────┼──────────────┤
│ VTS       │ 2010-01-26 07:41:00 │ 2010-01-26 07:45:00 │ … │        0.0 │          0.0 │          5.0 │
│ DDS       │ 2010-01-30 23:31:00 │ 2010-01-30 23:46:12 │ … │        0.0 │          0.0 │         16.3 │
│ DDS       │ 2010-01-18 20:22:20 │ 2010-01-18 20:38:12 │ … │        0.0 │          0.0 │         12.7 │
│ VTS       │ 2010-01-09 01:18:00 │ 2010-01-09 01:35:00 │ … │        0.0 │          0.0 │         14.3 │
│ CMT       │ 2010-01-18 19:10:14 │ 2010-01-18 19:17:07 │ … │       0.87 │          0.0 │         6.67 │
│ DDS       │ 2010-01-23 18:40:25 │ 2010-01-23 18:54:51

In [7]:
bah = rel1._rel

In [19]:
list(dw.query('select column_name from (describe from x)', x=bah).df()['column_name'])

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'rate_code',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount']

In [35]:
dw.query('select column_name from (describe from x)', x=rel1._rel).df()

,column_name
0,vendor_id
1,pickup_datetime
2,dropoff_datetime
3,passenger_count
4,trip_distance
5,pickup_longitude
6,pickup_latitude
7,rate_code
8,store_and_fwd_flag
9,dropoff_longitude


In [2]:
out = dw.Database(
    rel1 = 'data/yellow_tripdata_2010-01.parquet',
    # rel1 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet',
    rel2 = 'data/yellow_tripdata_2010-02.parquet',
)

In [5]:
out

Database:
    rel1: 'data/yellow_tripdata_2010-01.parquet'
    rel2: 'data/yellow_tripdata_2010-02.parquet'

In [16]:
out['rel1']

'data/yellow_tripdata_2010-01.parquet'

In [12]:
out2 = out >> 'pandas'

In [10]:
out2

Database:
    rel1: 14863778 x ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']
    rel2: 11145409 x ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', '__index_level_0__']

In [13]:
a = out2['rel1']

In [14]:
a

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2010-01-26 07:41:00,2010-01-26 07:45:00,1,0.75,-73.956778,40.767750,1,None,-73.965957,40.765232,CAS,4.5,0.0,0.5,0.00,0.0,5.00
1,DDS,2010-01-30 23:31:00,2010-01-30 23:46:12,1,5.90,-73.996118,40.763932,1,None,-73.981512,40.741193,CAS,15.3,0.5,0.5,0.00,0.0,16.30
2,DDS,2010-01-18 20:22:20,2010-01-18 20:38:12,1,4.00,-73.979673,40.783790,1,None,-73.917852,40.878560,CAS,11.7,0.5,0.5,0.00,0.0,12.70
3,VTS,2010-01-09 01:18:00,2010-01-09 01:35:00,2,4.70,-73.977922,40.763997,1,None,-73.923908,40.759725,CAS,13.3,0.5,0.5,0.00,0.0,14.30
4,CMT,2010-01-18 19:10:14,2010-01-18 19:17:07,1,0.60,-73.990924,40.734682,1,0,-73.995511,40.739088,Cre,5.3,0.0,0.5,0.87,0.0,6.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14863773,CMT,2010-01-06 14:51:19,2010-01-06 15:02:59,2,1.50,-73.970436,40.757928,1,0,-73.957113,40.764094,Cas,7.7,0.0,0.5,0.00,0.0,8.20
14863774,VTS,2010-01-09 12:52:00,2010-01-09 13:15:00,1,16.77,-73.784115,40.648632,1,None,-73.974673,40.750805,CAS,45.0,0.0,0.5,0.00,0.0,45.50
14863775,CMT,2010-01-09 14:00:44,2010-01-09 14:14:33,1,0.90,-74.002170,40.721619,1,0,-73.988528,40.721484,Cas,8.1,0.0,0.5,0.00,0.0,8.60
14863776,CMT,2010-01-09 09:52:23,2010-01-09 09:59:41,1,2.00,-73.980066,40.770469,1,0,-73.958075,40.783242,Cas,7.3,0.0,0.5,0.00,0.0,7.80


In [9]:
out >> 'pandas'

Database:
    rel1: 14863778 x ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']
    rel2: 11145409 x ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', '__index_level_0__']